In [1]:
import pandas as pd
d=pd.read_csv('online-valid.csv')
d.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,8049334,http://www.kueronakayaeotn.co.jp.kuerocekayaoa...,http://www.phishtank.com/phish_detail.php?phis...,2023-02-25T13:06:13+00:00,yes,2023-02-25T13:11:48+00:00,yes,Other
1,8049333,http://www.kueronakayaeotn.co.jp.kuerocekayaoa...,http://www.phishtank.com/phish_detail.php?phis...,2023-02-25T13:06:06+00:00,yes,2023-02-25T13:11:48+00:00,yes,Other
2,8049332,http://www.kueronakayaeotn.co.jp.kuerocekayaoa...,http://www.phishtank.com/phish_detail.php?phis...,2023-02-25T13:06:00+00:00,yes,2023-02-25T13:11:48+00:00,yes,Other
3,8049331,http://www.kueronakayaeotn.co.jp.kuerocekayaoa...,http://www.phishtank.com/phish_detail.php?phis...,2023-02-25T13:05:48+00:00,yes,2023-02-25T13:11:48+00:00,yes,Other
4,8049330,http://www.kueronakayaeotn.co.jp.kuerocekayaoa...,http://www.phishtank.com/phish_detail.php?phis...,2023-02-25T13:05:32+00:00,yes,2023-02-25T13:11:48+00:00,yes,Other


In [2]:
d.shape

(28335, 8)

In [7]:
phish=d.sample(n=1, random_state=12).copy()
phish=phish.reset_index(drop=True)
phish.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,8028082,https://lifemile.com.co/,http://www.phishtank.com/phish_detail.php?phis...,2023-02-06T18:59:00+00:00,yes,2023-02-06T19:03:22+00:00,yes,Other


In [9]:
phish.shape

(1, 8)

In [10]:
d1=pd.read_csv('Benign_list_big_final.csv')
d1.columns = ['URLs']
d1.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [11]:
legit = d1.sample(n=1, random_state=12).copy()
legit = legit.reset_index(drop=True)
legit.head()

,URLs
0,http://graphicriver.net/search?date=this-month...


In [12]:
legit.shape

(1, 1)

In [13]:
from urllib.parse import urlparse, urlencode
import ipaddress
import re

In [16]:
def getDomain(url):
    domain = urlparse(url).netloc
    if re.match(r"^www.",domain):
        domain = domain.replace("www.","")
    return domain
def havingIp(url):
    try:
        ipaddress.ip_address(url)
        ip = 1
    except:
        ip = 0
    return ip
def haveAtSign(url):
    if "@" in url:
        at = 1
    else:
        at = 0
    return at
def getLength(url):
    if len(url) < 54:
        length = 0
    else:
        length = 1
    return length
def getDepth(url):
    s = urlparse(url).path.split('/')
    depth = 0
    for j in range(len(s)):
        if len(s[j]) != 0:
            depth = depth+1
    return depth
def getRedirection(url):
    pos = url.rfind('//')
    if pos > 6:
        if pos > 7:
            return 1
        else:
            return 0
    else:
        return 0
def httpDomain(url):
    domain = urlparse(url).netloc
    if 'https' in domain:
        return 1
    else:
        return 0
def getSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1
    else:
        return 0


In [21]:
def featureExtraction(url, label):
    features = []
    features.append(getDomain(url))
    features.append(havingIp(url))
    features.append(haveAtSign(url))
    features.append(getLength(url))
    features.append(getDepth(url))
    features.append(getRedirection(url))
    features.append(httpDomain(url))
    features.append(getSuffix(url))
    features.append(label)
    return features

In [22]:
phish_features = []
label = 1
for i in range(0, 1):
    url = phish['url'][i]
    phish_features.append(featureExtraction(url, label))
    

In [23]:
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth', 'Redirection', 'https_Domain', 'Prefix/Suffix', 'Label']
phishing = pd.DataFrame(phish_features, columns= feature_names)
phishing.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,Prefix/Suffix,Label
0,lifemile.com.co,0,0,0,0,0,0,0,1


In [24]:
phishing.to_csv('phishing.csv', index=False)


In [26]:
legit_features = []
label = 0
for i in range(0,1):
    url = legit['URLs'][i]
    legit_features.append(featureExtraction(url, label))
features_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth', 'Redirection', 'https_Domain', 'Prefix/Suffix', 'Label']
legitimate = pd.DataFrame(legit_features, columns=features_names)
legitimate.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,Prefix/Suffix,Label
0,graphicriver.net,0,0,1,1,0,0,0,0


In [27]:
legitimate.to_csv('legitimate.csv', index=False)

In [28]:
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)
urldata.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,Prefix/Suffix,Label
0,graphicriver.net,0,0,1,1,0,0,0,0
1,lifemile.com.co,0,0,0,0,0,0,0,1


In [29]:
urldata.tail()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,Prefix/Suffix,Label
0,graphicriver.net,0,0,1,1,0,0,0,0
1,lifemile.com.co,0,0,0,0,0,0,0,1


In [30]:
urldata.shape

(2, 9)

In [31]:
urldata.to_csv('urldata.csv', index=False)